In [1]:
import pandas as pd
import numpy as np

In [2]:
titanic = pd.read_csv('C:\\Users\\Mark\\Desktop\\CDA\\Titanic\\all\\train.csv')

# 数据理解

In [3]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
titanic.describe() # Age 和 Fare 数值变化幅度很大，对于逻辑回归，可能会不收敛

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


# 数据清洗

In [6]:
titanic['Age']= titanic['Age'].fillna(titanic['Age'].median())

In [7]:
titanic['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [8]:
titanic['Embarked'] = titanic['Embarked'].fillna('S')

In [9]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       891 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


# 特征工程

Sex

In [10]:
titanic.loc[titanic['Sex'] == 'male','Sex'] = 1
titanic.loc[titanic['Sex'] == 'female','Sex'] = 0

In [11]:
titanic['Sex'].head()

0    1
1    0
2    0
3    0
4    1
Name: Sex, dtype: object

Embarked，use one-hot

In [12]:
Embarkeddf = pd.get_dummies(titanic['Embarked'], prefix = 'Embarked')
Embarkeddf.head()

,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1


In [13]:
titanic = pd.concat([titanic,Embarkeddf], axis = 1)

In [14]:
titanic.drop('Embarked', axis = 1, inplace = True) # axis =1 表示列，若没有，则默认删除行。inplace = True 表示原列所对应的内存值也随之改变
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,0,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,1,0,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,0,0,1
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,0,0,1


Pclass

In [15]:
Pclassdf = pd.get_dummies(titanic['Pclass'],prefix = 'Pclass') 
Pclassdf.head()    # prefix = 'Pclass' 总字段命名 Pclass，内表字段命名为Pclass_1，Pclass_2，Pclass_3

,Pclass_1,Pclass_2,Pclass_3
0,0,0,1
1,1,0,0
2,0,0,1
3,1,0,0
4,0,0,1


In [16]:
titanic = pd.concat([titanic, Pclassdf], axis = 1)

In [17]:
titanic.drop('Pclass', axis =1, inplace=True)
titanic.head()

,PassengerId,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,1,0,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,0,0,1,0,0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,1,0,0,1,0,0
2,3,1,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,0,1,0,0,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,0,0,1,1,0,0
4,5,0,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,0,0,1,0,0,1


Name

In [18]:
titanic['Name'].head()

0                              Braund, Mr. Owen Harris
1    Cumings, Mrs. John Bradley (Florence Briggs Th...
2                               Heikkinen, Miss. Laina
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                             Allen, Mr. William Henry
Name: Name, dtype: object

In [19]:
def getTitle(name):
    str1=name.split( ',' )[1] #Mr. Owen Harris
    str2=str1.split( '.' )[0]#Mr
    #strip() 方法用于移除字符串头尾指定的字符（默认为空格）
    str3=str2.strip()
    return str3

In [20]:
titledf = pd.DataFrame()
titledf['Title'] = titanic['Name'].apply(getTitle) # apply 和 map 作用一样


In [21]:
titledf['Title'].head()

0      Mr
1     Mrs
2    Miss
3     Mrs
4      Mr
Name: Title, dtype: object

In [22]:
title_mapDict = {
                    "Capt":       "Officer",
                    "Col":        "Officer",
                    "Major":      "Officer",
                    "Jonkheer":   "Royalty",
                    "Don":        "Royalty",
                    "Sir" :       "Royalty",
                    "Dr":         "Officer",
                    "Rev":        "Officer",
                    "the Countess":"Royalty",
                    "Dona":       "Royalty",
                    "Mme":        "Mrs",
                    "Mlle":       "Miss",
                    "Ms":         "Mrs",
                    "Mr" :        "Mr",
                    "Mrs" :       "Mrs",
                    "Miss" :      "Miss",
                    "Master" :    "Master",
                    "Lady" :      "Royalty"
                    }

In [23]:
titledf['Title'] = titledf['Title'].map(title_mapDict)  # 这里的map不能被apply代替

In [24]:
titledf['Title'].head()

0      Mr
1     Mrs
2    Miss
3     Mrs
4      Mr
Name: Title, dtype: object

In [25]:
titledf = pd.get_dummies(titledf['Title'])
titledf.head()

,Master,Miss,Mr,Mrs,Officer,Royalty
0,0,0,1,0,0,0
1,0,0,0,1,0,0
2,0,1,0,0,0,0
3,0,0,0,1,0,0
4,0,0,1,0,0,0


In [26]:
titanic = pd.concat([titanic, titledf], axis =1)

In [27]:
titanic.drop('Name', axis = 1, inplace=True)
titanic.head()

,PassengerId,Survived,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_C,...,Embarked_S,Pclass_1,Pclass_2,Pclass_3,Master,Miss,Mr,Mrs,Officer,Royalty
0,1,0,1,22.0,1,0,A/5 21171,7.2500,NaN,0,...,1,0,0,1,0,0,1,0,0,0
1,2,1,0,38.0,1,0,PC 17599,71.2833,C85,1,...,0,1,0,0,0,0,0,1,0,0
2,3,1,0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,...,1,0,0,1,0,1,0,0,0,0
3,4,1,0,35.0,1,0,113803,53.1000,C123,0,...,1,1,0,0,0,0,0,1,0,0
4,5,0,1,35.0,0,0,373450,8.0500,NaN,0,...,1,0,0,1,0,0,1,0,0,0


FamilySize

In [28]:
titanic['Parch'].value_counts()

0    678
1    118
2     80
5      5
3      5
4      4
6      1
Name: Parch, dtype: int64

In [29]:
titanic['SibSp'].value_counts()

0    608
1    209
2     28
4     18
3     16
8      7
5      5
Name: SibSp, dtype: int64

In [30]:
familyDf = pd.DataFrame()
familyDf[ 'FamilySize' ] = titanic[ 'Parch' ] + titanic[ 'SibSp' ] + 1
familyDf[ 'Family_Single' ] = familyDf[ 'FamilySize' ].map( lambda s : 1 if s == 1 else 0 )
familyDf[ 'Family_Small' ]  = familyDf[ 'FamilySize' ].map( lambda s : 1 if 2 <= s <= 4 else 0 )
familyDf[ 'Family_Large' ]  = familyDf[ 'FamilySize' ].map( lambda s : 1 if 5 <= s else 0 )
titanic = pd.concat([titanic,familyDf], axis = 1)
titanic.head()

,PassengerId,Survived,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_C,...,Master,Miss,Mr,Mrs,Officer,Royalty,FamilySize,Family_Single,Family_Small,Family_Large
0,1,0,1,22.0,1,0,A/5 21171,7.2500,NaN,0,...,0,0,1,0,0,0,2,0,1,0
1,2,1,0,38.0,1,0,PC 17599,71.2833,C85,1,...,0,0,0,1,0,0,2,0,1,0
2,3,1,0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,...,0,1,0,0,0,0,1,1,0,0
3,4,1,0,35.0,1,0,113803,53.1000,C123,0,...,0,0,0,1,0,0,2,0,1,0
4,5,0,1,35.0,0,0,373450,8.0500,NaN,0,...,0,0,1,0,0,0,1,1,0,0


# 选择特征

In [31]:
corrdf= titanic.corr()
corrdf

,PassengerId,Survived,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,...,Master,Miss,Mr,Mrs,Officer,Royalty,FamilySize,Family_Single,Family_Small,Family_Large
PassengerId,1.000000,-0.005007,0.034212,-0.057527,-0.001652,0.012658,-0.001205,-0.033606,0.022204,0.034303,...,-0.026151,-0.067846,0.038850,0.010197,0.055299,0.031602,-0.040143,0.057462,-0.028976,-0.057055
Survived,-0.005007,1.000000,-0.064910,-0.035322,0.081629,0.257307,0.168240,0.003650,-0.149683,0.285904,...,0.085221,0.332795,-0.549199,0.344935,-0.031316,0.033391,0.016639,-0.203367,0.279855,-0.125147
Age,0.034212,-0.064910,1.000000,-0.233296,-0.172482,0.096688,0.030248,-0.031415,-0.006729,0.323896,...,-0.373960,-0.248767,0.180808,0.166798,0.179927,0.070654,-0.245619,0.171647,-0.068249,-0.204206
SibSp,-0.057527,-0.035322,-0.233296,1.000000,0.414838,0.159651,-0.059528,-0.026354,0.068734,-0.054582,...,0.349559,0.084945,-0.250489,0.059941,-0.024712,-0.008384,0.890712,-0.584471,0.213225,0.730691
Parch,-0.001652,0.081629,-0.172482,0.414838,1.000000,0.216225,-0.011069,-0.081228,0.060814,-0.017633,...,0.267344,0.102514,-0.333905,0.221318,-0.048211,-0.035583,0.783111,-0.583398,0.265863,0.631523
Fare,0.012658,0.257307,0.096688,0.159651,0.216225,1.000000,0.269335,-0.117216,-0.162184,0.591711,...,0.010908,0.120829,-0.183766,0.105665,0.010357,0.015044,0.217138,-0.271832,0.205527,0.143636
Embarked_C,-0.001205,0.168240,0.030248,-0.059528,-0.011069,0.269335,1.000000,-0.148258,-0.782742,0.296423,...,-0.035225,0.037613,-0.072567,0.066101,-0.008034,0.079020,-0.046215,-0.095298,0.158586,-0.109274
Embarked_Q,-0.033606,0.003650,-0.031415,-0.026354,-0.081228,-0.117216,-0.148258,1.000000,-0.499421,-0.155342,...,0.010478,0.168720,-0.078338,-0.091121,0.012618,-0.023105,-0.058592,0.086464,-0.087093,-0.005620
Embarked_S,0.022204,-0.149683,-0.006729,0.068734,0.060814,-0.162184,-0.782742,-0.499421,1.000000,-0.161921,...,0.024264,-0.139126,0.112870,-0.000565,-0.000902,-0.054685,0.077359,0.029074,-0.084120,0.099265
Pclass_1,0.034303,0.285904,0.323896,-0.054582,-0.017633,0.591711,0.296423,-0.155342,-0.161921,1.000000,...,-0.084700,0.021958,-0.097288,0.091483,0.104919,0.132798,-0.046114,-0.113364,0.168568,-0.092945


In [32]:
corrdf['Survived'].sort_values(ascending = False)

Survived         1.000000
Mrs              0.344935
Miss             0.332795
Pclass_1         0.285904
Family_Small     0.279855
Fare             0.257307
Embarked_C       0.168240
Pclass_2         0.093349
Master           0.085221
Parch            0.081629
Royalty          0.033391
FamilySize       0.016639
Embarked_Q       0.003650
PassengerId     -0.005007
Officer         -0.031316
SibSp           -0.035322
Age             -0.064910
Family_Large    -0.125147
Embarked_S      -0.149683
Family_Single   -0.203367
Pclass_3        -0.322308
Mr              -0.549199
Name: Survived, dtype: float64

# Test

In [33]:
test = pd.read_csv('C:\\Users\\Mark\\Desktop\\CDA\\Titanic\\all\\test.csv')

In [34]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [35]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


# 数据清洗

Age

In [36]:
test['Age'] = test['Age'].fillna(test['Age'].mean())

Fare

In [37]:
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())

In [38]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           418 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


# 特征工程

In [39]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


Sex

In [40]:
test.loc[test['Sex'] == 'male','Sex'] = 1
test.loc[test['Sex'] == 'female', 'Sex'] =0 

In [41]:
test['Sex'].head()

0    1
1    0
2    1
3    1
4    0
Name: Sex, dtype: object

Pclass

In [42]:
Pclasstdf = pd.get_dummies(test['Pclass'], prefix = 'Pclass')
Pclasstdf.head()

,Pclass_1,Pclass_2,Pclass_3
0,0,0,1
1,0,0,1
2,0,1,0
3,0,0,1
4,0,0,1


In [43]:
test = pd.concat([test,Pclasstdf], axis = 1)

In [44]:
test.drop('Pclass', axis = 1, inplace = True)
test.head()

,PassengerId,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Pclass_1,Pclass_2,Pclass_3
0,892,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,NaN,Q,0,0,1
1,893,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,NaN,S,0,0,1
2,894,"Myles, Mr. Thomas Francis",1,62.0,0,0,240276,9.6875,NaN,Q,0,1,0
3,895,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,NaN,S,0,0,1
4,896,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,3101298,12.2875,NaN,S,0,0,1


Embarked

In [45]:
Embarkedtdf = pd.get_dummies(test['Embarked'], prefix='Embarked')
Embarkedtdf.head()

,Embarked_C,Embarked_Q,Embarked_S
0,0,1,0
1,0,0,1
2,0,1,0
3,0,0,1
4,0,0,1


In [46]:
test = pd.concat([test,Embarkedtdf], axis =1)
test.drop('Embarked', axis =1, inplace = True)
test.head()

,PassengerId,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,892,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,NaN,0,0,1,0,1,0
1,893,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,NaN,0,0,1,0,0,1
2,894,"Myles, Mr. Thomas Francis",1,62.0,0,0,240276,9.6875,NaN,0,1,0,0,1,0
3,895,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,NaN,0,0,1,0,0,1
4,896,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,3101298,12.2875,NaN,0,0,1,0,0,1


Name

In [47]:
titletdf = pd.DataFrame()
titletdf = test['Name'].map(getTitle)
titletdf.head()

0     Mr
1    Mrs
2     Mr
3     Mr
4    Mrs
Name: Name, dtype: object

In [48]:
titletdf = titletdf.map(title_mapDict)
titletdf.head()

0     Mr
1    Mrs
2     Mr
3     Mr
4    Mrs
Name: Name, dtype: object

In [49]:
titletdf = pd.get_dummies(titletdf)
titletdf.head()

,Master,Miss,Mr,Mrs,Officer,Royalty
0,0,0,1,0,0,0
1,0,0,0,1,0,0
2,0,0,1,0,0,0
3,0,0,1,0,0,0
4,0,0,0,1,0,0


In [50]:
test = pd.concat([test,titletdf], axis =1)
test.drop('Name', axis = 1, inplace = True)
test.head()

,PassengerId,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Master,Miss,Mr,Mrs,Officer,Royalty
0,892,1,34.5,0,0,330911,7.8292,NaN,0,0,1,0,1,0,0,0,1,0,0,0
1,893,0,47.0,1,0,363272,7.0000,NaN,0,0,1,0,0,1,0,0,0,1,0,0
2,894,1,62.0,0,0,240276,9.6875,NaN,0,1,0,0,1,0,0,0,1,0,0,0
3,895,1,27.0,0,0,315154,8.6625,NaN,0,0,1,0,0,1,0,0,1,0,0,0
4,896,0,22.0,1,1,3101298,12.2875,NaN,0,0,1,0,0,1,0,0,0,1,0,0


familySize

In [51]:
familytDf = pd.DataFrame()
familytDf[ 'FamilySize' ] = test[ 'Parch' ] + test[ 'SibSp' ] + 1
familytDf[ 'Family_Single' ] = familytDf[ 'FamilySize' ].map( lambda s : 1 if s == 1 else 0 )
familytDf[ 'Family_Small' ]  = familytDf[ 'FamilySize' ].map( lambda s : 1 if 2 <= s <= 4 else 0 )
familytDf[ 'Family_Large' ]  = familytDf[ 'FamilySize' ].map( lambda s : 1 if 5 <= s else 0 )
test = pd.concat([test,familytDf], axis =1)
test.head()

,PassengerId,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Pclass_1,Pclass_2,...,Master,Miss,Mr,Mrs,Officer,Royalty,FamilySize,Family_Single,Family_Small,Family_Large
0,892,1,34.5,0,0,330911,7.8292,NaN,0,0,...,0,0,1,0,0,0,1,1,0,0
1,893,0,47.0,1,0,363272,7.0000,NaN,0,0,...,0,0,0,1,0,0,2,0,1,0
2,894,1,62.0,0,0,240276,9.6875,NaN,0,1,...,0,0,1,0,0,0,1,1,0,0
3,895,1,27.0,0,0,315154,8.6625,NaN,0,0,...,0,0,1,0,0,0,1,1,0,0
4,896,0,22.0,1,1,3101298,12.2875,NaN,0,0,...,0,0,0,1,0,0,3,0,1,0


In [52]:
source_X = pd.concat([titanic['Fare'], titledf, Embarkeddf, Pclassdf, familyDf,titanic['Sex']], axis=1)

In [53]:
source_Y = titanic['Survived']

In [54]:
pred_X = pd.concat([test['Fare'], titletdf, Embarkedtdf, Pclasstdf, familytDf,test['Sex']], axis=1)

In [55]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.cross_validation import train_test_split
train_X, test_X, train_y, test_y = train_test_split(source_X ,
                                                    source_Y,
                                                    train_size=.8)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [56]:
print ('原始数据集特征：',source_X.shape, 
       '训练数据集特征：',train_X.shape ,
      '测试数据集特征：',test_X.shape)

print ('原始数据集标签：',source_Y.shape, 
       '训练数据集标签：',train_y.shape ,
      '测试数据集标签：',test_y.shape)

原始数据集特征： (891, 18) 训练数据集特征： (712, 18) 测试数据集特征： (179, 18)
原始数据集标签： (891,) 训练数据集标签： (712,) 测试数据集标签： (179,)


In [57]:
from sklearn.linear_model import LogisticRegression

In [58]:
model = LogisticRegression()
model.fit( train_X , train_y )

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [59]:
model.score(test_X , test_y )

0.8268156424581006

In [60]:
pred_Y = model.predict(pred_X)
pred_Y=pred_Y.astype(int)
#乘客id
passenger_id = test['PassengerId']
#数据框：乘客id，预测生存情况的值
predDf = pd.DataFrame( 
    { 'PassengerId': passenger_id , 
     'Survived': pred_Y } )
predDf.shape 

(418, 2)

In [61]:
#保存结果
predDf.to_csv( 'C:\\Users\\Mark\\Desktop\\CDA\\Titanic\\all\\titanic_pred.csv' , index = False )